In [17]:
!pip install keras==2.2.4

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import gc; gc.enable()
import pandas as pd
from torch.utils.data import Dataset
import tensorflow as tf
from tensorflow import keras
from torchvision import transforms

In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Input
# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

Using TensorFlow backend.


In [ ]:
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import cohen_kappa_score
# Specify title of our final model
SAVED_MODEL_NAME = 'resnet_model.h5'
def get_preds_and_labels(model, generator):
    """
    Get predictions and labels from the generator
    """
    preds = []
    labels = []
    for _ in range(int(np.ceil(generator.samples / batch_size))):
        x, y = next(generator)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel(), np.concatenate(labels).ravel()

class Metrics(Callback):
    """
    A custom Keras callback for saving the best model
    according to the Quadratic Weighted Kappa (QWK) metric
    """
    def on_train_begin(self, logs={}):
        """
        Initialize list of QWK scores on validation data
        """
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        """
        Gets QWK score on the validation data
        """
        # Get predictions and convert to integers
        y_pred, labels = get_preds_and_labels(model, val_generator)
        y_pred = np.rint(y_pred).astype(np.uint8).clip(0, 4)
        # We can use sklearns implementation of QWK straight out of the box
        # as long as we specify weights as 'quadratic'
        _val_kappa = cohen_kappa_score(labels, y_pred, weights='quadratic')
        self.val_kappas.append(_val_kappa)
        print(f"val_kappa: {round(_val_kappa, 4)}")
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save(SAVED_MODEL_NAME)
        return

In [4]:
train_dir = '/Users/flatironschool/Documents/Kaggle/Kaggle-DR-detection/data/train/'



In [5]:
from keras.layers import Dense
#instantiate pre-trained ResNet model
res_fifty = keras.applications.ResNet50(include_top=False, weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


/Users/flatironschool/.local/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [6]:
input_shape = res_fifty.output_shape
out = res_fifty.output
out = keras.layers.GlobalAveragePooling2D()(out)
out = keras.layers.Dense(512, activation='relu')(out)
out = keras.layers.Dense(512, activation='relu')(out)
total_classes = 5
predictions = keras.layers.Dense(total_classes, activation='softmax')(out)

model = keras.models.Model(inputs=res_fifty.input, outputs=predictions)

In [7]:
p = 179//4
cnt = 0
set_trainable = False
for layer in model.layers:
    if cnt>p:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
    cnt+=1

In [12]:
pd.set_option('max_colwidth', -1)

layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
df_layers = pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])
df_layers

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x148ebdb00>,input_1,False
1,<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x148ebde10>,conv1_pad,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x10fc5b4a8>,conv1,False
3,<tensorflow.python.keras.layers.normalization.BatchNormalizationV1 object at 0x110babe80>,bn_conv1,False
4,<tensorflow.python.keras.layers.core.Activation object at 0x148ed1390>,activation,False
5,<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x14a793320>,pool1_pad,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x14a793278>,max_pooling2d,False
7,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x14a7adb70>,res2a_branch2a,False
8,<tensorflow.python.keras.layers.normalization.BatchNormalizationV1 object at 0x14a85d550>,bn2a_branch2a,False
9,<tensorflow.python.keras.layers.core.Activation object at 0x14a85def0>,activation_1,False


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import optimizers
datagen = keras.preprocessing.image.ImageDataGenerator(validation_split = 0.15)
batch_size = 32
train_steps_per_epoch = len(os.listdir(train_dir))//batch_size
adm = tf.keras.optimizers.Adam(lr = 0.0001)

train_generator = datagen.flow_from_directory(train_dir,
                                              target_size = (224,224),
                                              batch_size = batch_size,
                                              class_mode ='categorical') 
val_generator = datagen.flow_from_directory(train_dir, 
                                            target_size=(224, 224),
                                            batch_size=batch_size,
                                            class_mode='categorical')
# For tracking Quadratic Weighted Kappa score
kappa_metrics = Metrics()
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=10)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=3, 
                        verbose=1, 
                        mode='auto', 
                        epsilon=0.0001)

model.compile(optimizer = adm, loss = 'categorical_crossentropy')
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch, 
                              validation_data=val_generator,
                              validation_steps = val_generator.samples // batch_size,
                              epochs=150, verbose=1, callbacks=[kappa_metrics, es, rlr])

Found 24941 images belonging to 5 classes.
Found 24941 images belonging to 5 classes.
Epoch 1/150
780/780 [==============================] - 4276s 5s/step - loss: 1.5764
val_kappa: 0.4968
Validation Kappa has improved. Saving model.
780/780 [==============================] - 24921s 32s/step - loss: 0.3046 - val_loss: 1.5764
Epoch 2/150
780/780 [==============================] - 4261s 5s/step - loss: 1.6843
val_kappa: 0.4921
780/780 [==============================] - 24888s 32s/step - loss: 0.1779 - val_loss: 1.6843
Epoch 3/150
780/780 [==============================] - 4238s 5s/step - loss: 2.4096
val_kappa: 0.5288
Validation Kappa has improved. Saving model.
780/780 [==============================] - 24816s 32s/step - loss: 0.1108 - val_loss: 2.4096
Epoch 4/150
780/780 [==============================] - 7911s 10s/step - loss: 2.8109


In [8]:
datagen = keras.preprocessing.image.ImageDataGenerator()
train_generator = datagen.flow_from_directory(train_dir, target_size = (224,224),class_mode ='categorical') 
batch_size = 32
train_steps_per_epoch = len(os.listdir(train_dir))//batch_size
adm = tf.keras.optimizers.Adam(lr = 0.0001)
model.compile(optimizer = adm, loss = 'categorical_crossentropy')
history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch,
                              epochs=15, verbose=1)

Found 16174 images belonging to 5 classes.
Epoch 1/150
506/506 [==============================] - 4246s 8s/step - loss: 0.5413
Epoch 2/150
506/506 [==============================] - 4295s 8s/step - loss: 0.2573
Epoch 3/150
506/506 [==============================] - 4446s 9s/step - loss: 0.1418
Epoch 4/150
506/506 [==============================] - 5655s 11s/step - loss: 0.1017
Epoch 5/150
506/506 [==============================] - 58591s 116s/step - loss: 0.0744
Epoch 6/150
 42/506 [=>............................] - ETA: 1:08:13 - loss: 0.0531

KeyboardInterrupt: 

In [9]:
plt.plot(history.history['loss'])
plt.savefig('loss_resnet50_20190911')

NameError: name 'history' is not defined

In [10]:
model_json = model.to_json()
with open("res_fifty.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("res_fifty.h5")
print("Saved model to disk")

Saved model to disk


In [20]:
train_features = np.zeros(shape=(16174, 512))
train_labels = np.zeros(shape=(16174,5))

In [23]:
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg19.predict(inputs_batch)
    train_features[i : i + len(inputs_batch)] = features_batch
    train_labels[i : i +len(inputs_batch)] = labels_batch
    i += len(inputs_batch)
    print(i)
    if i+2 > len(imgs):
        break
         
train_features = np.reshape(train_features, (len(imgs), 7 * 7 * 512))

NameError: name 'vgg19' is not defined